In [17]:
import duckdb

In [18]:
con = duckdb.connect('phish_pipeline.duckdb')
con.sql('select * from information_schema.tables')


┌────────────────┬──────────────┬───────────────────────┬────────────┬──────────────────────────────┬──────────────────────┬───────────────────────────┬──────────────────────────┬────────────────────────┬────────────────────┬──────────┬───────────────┬───────────────┐
│ table_catalog  │ table_schema │      table_name       │ table_type │ self_referencing_column_name │ reference_generation │ user_defined_type_catalog │ user_defined_type_schema │ user_defined_type_name │ is_insertable_into │ is_typed │ commit_action │ TABLE_COMMENT │
│    varchar     │   varchar    │        varchar        │  varchar   │           varchar            │       varchar        │          varchar          │         varchar          │        varchar         │      varchar       │ varchar  │    varchar    │    varchar    │
├────────────────┼──────────────┼───────────────────────┼────────────┼──────────────────────────────┼──────────────────────┼───────────────────────────┼──────────────────────────┼──────────────

In [19]:
# just an example of how to query the tables
setlists  = con.sql('select * from phish_data.setlists')
setlists

┌────────────┬────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┬──────────┬────────────────┬─────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
sql = '''
create or replace view user_setlists_v as
(
with 
setlists as
(
SELECT
	showid,
	showdate,
	songid,
	position,
	set,
	tourname,
	tourwhen,
	artist_name,
	song
FROM
	phish_data.setlists
),

attended as
(
select
	uid,
	username,
	showid,
	venueid,
	venue,
	city,
	state,
	artist_name
from
	phish_data.user_attendance__data
)
SELECT *
FROM 
setlists
LEFT join
attended
	using (showid,
artist_name)
)
'''

con.sql(sql)

In [21]:
user_setlists = con.sql('select * from user_setlists_v')
user_setlists.count('*')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     51955006 │
└──────────────┘

In [24]:
user_setlists.limit(10)

┌────────────┬────────────┬─────────┬──────────┬─────────┬───────────────────────────────────────┬─────────────┬─────────────┬────────────────────────────────┬─────────┬──────────┬─────────┬────────────────────────┬───────────────┬─────────┐
│   showid   │  showdate  │ songid  │ position │   set   │               tourname                │  tourwhen   │ artist_name │              song              │   uid   │ username │ venueid │         venue          │     city      │  state  │
│  varchar   │  varchar   │ varchar │ varchar  │ varchar │                varchar                │   varchar   │   varchar   │            varchar             │ varchar │ varchar  │ varchar │        varchar         │    varchar    │ varchar │
├────────────┼────────────┼─────────┼──────────┼─────────┼───────────────────────────────────────┼─────────────┼─────────────┼────────────────────────────────┼─────────┼──────────┼─────────┼────────────────────────┼───────────────┼─────────┤
│ 1569515725 │ 2019-12-30 │ 466 

In [22]:
# con.close()